In [94]:
# Import thư viện
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.svm import SVC, SVR
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle

# Data Preparation

In [ ]:
# Load Input
df = pd.read_csv(r'D:\Pomona\dataset\cleaned_dataset.csv') 

# Create Label Columns
df['type_code'] = df["file_name"].str[22:24]
df['status_code'] = df["file_name"].str[26:28]
data = df

# Choose Meat Type data to train: "01" - Beef; "02" - Pork.
data = data[data["type_code"] == "01"]

# Data
X = data.loc[:, 'sample_signal_1':'sample_signal_228']

# remap label: 1,2,3,5 -> 0,1,2,3
y = data['status_code'].astype('category').cat.codes

# Data Spliting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [73]:
svm_freshness = SVC(kernel='linear')
svm_freshness.fit(X_train, y_train)
y_fresh_pred = svm_freshness.predict(X_test)
print('SVM: ')
print(classification_report(y_test, y_fresh_pred))

SVM: 
              precision    recall  f1-score   support

           0       0.88      0.91      0.90       437
           1       0.92      0.90      0.91       547
           2       0.91      0.92      0.92       545
           3       0.90      0.85      0.88       238

    accuracy                           0.90      1767
   macro avg       0.90      0.90      0.90      1767
weighted avg       0.90      0.90      0.90      1767



In [74]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print('Random Forest: ')
print(classification_report(y_test, y_pred_rf))

Random Forest: 
              precision    recall  f1-score   support

           0       0.84      0.70      0.76       437
           1       0.83      0.96      0.89       547
           2       0.89      0.89      0.89       545
           3       0.87      0.84      0.85       238

    accuracy                           0.86      1767
   macro avg       0.86      0.85      0.85      1767
weighted avg       0.86      0.86      0.85      1767



In [75]:
xgb_status = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_status.fit(X_train, y_train)
y_pred_status = xgb_status.predict(X_test)
print('XGB: ')
print(classification_report(y_test, y_pred_status))

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:28:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGB: 
              precision    recall  f1-score   support

           0       0.89      0.79      0.84       437
           1       0.88      0.97      0.93       547
           2       0.92      0.92      0.92       545
           3       0.92      0.89      0.91       238

    accuracy                           0.90      1767
   macro avg       0.90      0.89      0.90      1767
weighted avg       0.90      0.90      0.90      1767



# Save Model

In [90]:
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm_freshness, f)
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)
with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(xgb_status, f)

# Load Model

In [91]:
with open('svm_model.pkl', 'rb') as f:
    svm_model = pickle.load(f)
with open('rf_model.pkl', 'rb') as f:
    rf_model = pickle.load(f)
with open('xgb_model.pkl', 'rb') as f:
    xgb_model = pickle.load(f)

# Predict 

In [92]:
predict1 = svm_model.predict(X_test)
predict2 = rf_model.predict(X_test)
predict3 = xgb_model.predict(X_test)

In [93]:
print(predict1)
print(predict2)
print(predict3)

[3 1 0 ... 1 3 0]
[3 1 0 ... 1 3 0]
[3 1 0 ... 1 0 0]
